In [1]:
import pandas as pd

In [3]:
df1 = pd.read_csv('theater_article.csv', thousands=',')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2405 entries, 0 to 2404
Data columns (total 5 columns):
title       2405 non-null object
link        2405 non-null object
contents    2405 non-null object
like        2405 non-null int64
angry       2405 non-null int64
dtypes: int64(2), object(3)
memory usage: 94.1+ KB


In [4]:
df1['emotion'] = df1['like'] - df1['angry']

In [5]:
# 감정(좋아요 - 싫어요)가 0보다 크면 가격 상승, 아니면 가격하락
def emo_preprocessing(text):
    value = int(text)
    if value > 0 :
        return '1'
    else :
        return '0'

In [6]:
df1['emotion'] = df1['emotion'].apply(emo_preprocessing)
df1['emotion'].value_counts()

0    1546
1     859
Name: emotion, dtype: int64

In [7]:
X = df1['contents']
y = df1['emotion']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [11]:
from konlpy.tag import *

# kkma = Kkma()
okt = Okt()

In [12]:
def tokenizer(text):
    return okt.morphs(text)

In [13]:
tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)
model1 = LogisticRegression()
pipe = Pipeline([('vect', tfidf),('clf', model1)])

In [14]:
pipe.fit(X_train, y_train)

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...\w\\w+\\b',
                                 tokenizer=<function tokenizer at 0x00000234AE757828>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_

In [15]:
# 학습 정확도 측정
y_pred = pipe.predict(X_test)
print(f'정확도 : {accuracy_score(y_test, y_pred)}')

정확도 : 0.6993355481727574


In [16]:
import pickle

In [17]:
with open('theater_article_pipe.dat', 'wb') as fp :
    pickle.dump(pipe, fp)